In [13]:
# 고객센터 - 
import os
import json
from openai import OpenAI
client = OpenAI()

# 가상의 고객 데이터
customer_reviews = [
    {'id' : 1,
     'product' : '무선 청소기',
     'date' : '2025-01-10',
     'content' : '너무 시끄럽고 충전이 잘 안되요 그냥 환불 하고 싶습니다. 박스는 버렸어요'
     },
    {'id' : 2,
     'product' : '여름용 셔츠',
     'date' : '2025-02-10',
     'content' : '사이즈가 광고랑 달라요 배송도 느리고 교환 가능한가요? 구매한지는 한달이 조금 넘었습니다.'
     },
    {'id' : 3,
     'product' : '블루투스 키보드',
     'date' : '2025-03-10',
     'content' : '블루투스가 잘 안잡혀요. 아니 인식이 안되는거 같아요.. 교환 또는 환불해주세요. 박스는 있어요'
     },
]
# 분류 - 판단 - 대응
# 회사의 환불/보상 규정( Knowledge Base for RAG)
policy_doc = '''
[다판다 교환/환불 및 보상 규정]
1. 단순 변심 환불 : 구매후 7일 이내 가능, 단 제품 박스가 온전해야 함
2. 제품 불량 환불 : 구매후 30일 이내 가능, 박스유무 상관 없음
3. 불만족 보상 : 구매 확정후에는 환불 불가, 소정의 적립금(2,000원) 지급 가능
4. AS 규정 : 구매 후 1년 이내 무상 AS 가능
'''

def ask_gpt_json(prompt):
    response = client.chat.completions.create(model = 'gpt-5-nano',
                                   messages=[{'role':'user','content':prompt}],
                                   response_format={'type':'json_object'}
                                   )
    try:
        return response.choices[0].message.content
    except:
        return None

In [14]:
# 단순히 요약이 아닌 카테고리, 감정점수, 핵심요구사항을 뽑아야함
def analyze_review(text):
    prompt = f'''
    너는 쇼핑몰 리뷰 전문가야, 아래 리뷰를 분석해서 다음 json 형식으로 출력
    
    [출력형식]
    {{
        'sentiment' : '긍정/부정/중립 중 하나',
        'category' : '배송/품질/서비스/기타 중 하나',
        'request_type' : '환불/교환/보상/AS/단순후기 중 하나',
        'key_issue' : "핵심 불만 사항(10자 이내)"
    }}

    [리뷰]
    {text}
    '''
    return json.loads(ask_gpt_json(prompt))  # 문자열 형태의 dict를 실제 객체로 변환

analzed_data_list = []
for item in customer_reviews:
    analysis = analyze_review( item['content'] )
    # 원본데이터에 분석 결과 합치기
    merged_data = {**item, **analysis}
    analzed_data_list.append(merged_data)
    

In [15]:
analzed_data_list

[{'id': 1,
  'product': '무선 청소기',
  'date': '2025-01-10',
  'content': '너무 시끄럽고 충전이 잘 안되요 그냥 환불 하고 싶습니다. 박스는 버렸어요',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '환불',
  'key_issue': '소음과충전불량'},
 {'id': 2,
  'product': '여름용 셔츠',
  'date': '2025-02-10',
  'content': '사이즈가 광고랑 달라요 배송도 느리고 교환 가능한가요? 구매한지는 한달이 조금 넘었습니다.',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '사이즈 불일치'},
 {'id': 3,
  'product': '블루투스 키보드',
  'date': '2025-03-10',
  'content': '블루투스가 잘 안잡혀요. 아니 인식이 안되는거 같아요.. 교환 또는 환불해주세요. 박스는 있어요',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '블루투스불량'}]

In [16]:
# ai가 멋대로 판단하지 않고 근거를 기반으로 생각하게 함
def make_decision(customer_data, policy_doc):
    prompt = f'''
        너는 베테랑 CS 매니저야
        아래 [고객 데이터]와 [회사규정]을 비교해서 요구사항을 들어줄 수 있는지 판단해

        [회사규정]
        {policy_doc}

        [고객데이터]
        - 구매일 : {customer_data['date']}
        - 오늘날짜 : 2025-11-24(기준)
        - 요청유형 : {customer_data['request_type']}
        - 내용 : {customer_data['content']}

        [지시사항]
        1. 규정의 몇 번 조항에 해당되는지 먼저 생각할 것(Cot)
        2. 날자를 계산해서 기간 내인지 확인할 것
        3. 최종 결정을 json으로 출력할 것

        [출력형식]
        {{
            'reasoning':'판단 근거(단계별 서술)',
            'decision':'승인/거절',
            'action_guide' : '상담원이 해야 할 행동 가이드'
        }}
    '''
    return json.loads(ask_gpt_json(prompt))

decision_results = []
for data in analzed_data_list:
    decision = make_decision(data,policy_doc)
    # 결과 저장
    final_result = {**data, 'decision_info':decision}
    decision_results.append(final_result)

    print(f'{data['id']} 판단결과')
    print(f'사유 : {decision['reasoning']}')
    print(f'결정 : {decision['decision']}')
    print(f'행동 가이드 : {decision['action_guide']}')
    print('-'*100)

1 판단결과
사유 : Step 1: 규정 조항 매칭
- 단순 변심 환불(조항 1): 박스가 온전해야 하므로 박스가 이미 버려진 현재 조건에서는 해당되지 않음.
- 제품 불량 환불(조항 2): 구매일로부터 30일 이내여야 하는데, 현재 날짜 기준으로 약 318일 경과로 불가.
- 불만족 보상(조항 3): 구매 확정 후에는 환불 불가, 대신 소정의 적립금 지급 가능(2,000원) 가능 여부가 있음.
- AS 규정(조항 4): 구매 후 1년 이내 무상 AS 가능.
관계 정리: 이번 요청은 환불이지만 규정에 따르면 즉시 환불은 거절되며, AS 가능성이 남아 있음. 또한 박스 있음 여부도 환불 요건에 영향을 주는 점 확인.
Step 2: 날짜 계산
- 구매일: 2025-01-10
- 오늘날짜: 2025-11-24
- 경과일: 약 318일 (1년 이내) → AS 대상 가능.
Step 3: 최종 판단
- 요청하신 환불은 규정상 승인 불가(박스 미보유, 30일 제한 초과). 대신 AS를 통한 점검/수리 가능성을 제시하고, 원하신다면 불만족 보상으로 2,000원 적립금 옵션도 제공 가능.
Step 4: 요약
- 현재 상황에서 환불은 승인 불가. AS를 통한 무상 수리/진단을 제안하고, 필요 시 2,000원 적립금으로 대체 보상도 가능.
결정 : 거절
행동 가이드 : 다음과 같이 안내한다. 
1) 정중하게 사과하고, 환불 불가 사유(박스 미보유, 구매후 7일/30일 기한 경과)를 간단히 설명합니다. 
2) 대체 옵션 제공: (a) 1년 이내 무상 AS 접수 및 진단/수리 진행 제안, (b) 불만족 보상으로 2,000원 적립금 지급 가능 여부 확인을 제시. 
3) AS 절차 안내: 주문번호/제품 모델/시리얼 번호 확인, 문제 현상(소음 정도, 충전 문제) 상세 기록, 배송/방문 서비스 선호도 확인, 수리 예상 일정 안내. 
4) 필요 정보 수집: 구매 증빙(영수증/주문번호), 주소/연락처, 현재 사용 환경. 
5) 고객 의사에 맞춰 다음 단계로 진행: AS 접수 양식 전달, 수리

In [17]:
# Prompt Chaining 기법   이전 단계의 reasoning을 이용
# 앞의 판단 결과에 따라서 (승인/거절) 톤앤매너를 다르게 해서 이메일을 작성

# 승인 : 친절하고 빠르게 안내
# 거절 : 단호하게 거절하되 공감하는 톤

def draft_email(final_data):
    # 정보추출
    product = final_data['product']
    customer_issue = final_data['content']
    decison = final_data['decision_info']['decision']
    reason = final_data['decision_info']['reasoning']
    tone = '매우 정중하고 사과하는 태도' if decision == '승인' else '안타까워하며 공감하지만 원칙을 지키는 태도'

    prompt = f'''
        고객에게 보낼 이메일 작성 해줘

        상황 : 고객이 {product}에 대해 {customer_issue} 라고 문의
        결정 : {decison}
        이유 : {reason}
        작성 톤 : {tone}

        이메일 형식:
        제목 : 문의하신 {product}건에 대한 답변입니다.
        내용 : (인사 - 공감 - 결정 내용 통보 - 이유 설명 - 마무리)
    '''
    response = client.chat.completions.create(
        model = 'gpt-5-nano',
        messages=[{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content
for res in decision_results:
    email = draft_email(res)
    print(email)
    print('-'*100)

제목: 문의하신 무선 청소기 건에 대한 답변입니다.

내용:
고객님 안녕하세요. 먼저 무선 청소기의 소음과 충전 문제가 지속되어 불편을 겪으신 점에 대해 진심으로 위로의 말씀을 드립니다. 상황을 확인한 결과, 아래와 같이 안내드리겠습니다.

- 결정 내용: 요청하신 환불은 현재 정책상 승인하기 어렵습니다.
- 이유 설명:
  - 박스 보유 여부: 박스를 이미 버리신 상태로 반품 요건 중 포장 상태 충족이 어렵습니다.
  - 반품 기간 요건: 구매일이 2025-01-10이고 오늘 날짜가 2025-11-24로, 구매 후 30일 이내의 불량 환불 요건은 경과했습니다(약 318일 경과).
  - AS 가능성: 다만 구매 후 1년 이내 무상 AS 대상이므로 AS를 통한 점검 및 수리 서비스가 가능합니다.
  - 보상 대안: 원하신다면 불만족 보상으로 2,000원 적립금을 지급하는 방법도 안내해 드릴 수 있습니다.

제안 드리는 해결 방법:
- AS 안내 및 접수: AS 접수 절차를 통해 기기 점검 및 무상 수리를 진행해 드리겠습니다. 접수 방법은 고객센터 문의나 온라인 접수 페이지를 이용해 주시면 됩니다. 접수 시 주문번호, 구매일, 시리얼 넘버, 증상 등을 준비해 주시면 원활하게 처리됩니다.
- 원하시면 2,000원 적립금 보상 선택 가능: 적립금 지급 여부와 지급 방법에 대해 안내해 드리겠습니다.

추가로 원하시는 해결 방식이나 문의사항이 있으면 언제든지 말씀해 주세요. 최선을 다해 도와드리겠습니다.

감사합니다.
[회사명] 고객지원팀
----------------------------------------------------------------------------------------------------
제목: 문의하신 여름용 셔츠건에 대한 답변입니다.

안녕하세요 고객님,

여름용 셔츠의 사이즈가 광고와 다르고 배송이 지연되어 불편을 겪으셨다는 점에 대해 진심으로 공감합니다. 이로 인해 실망을 드려 정말 죄송합니다.

다만 이번 건에 대해서는 교환 승인을